A partir da execução desse notebook, será gerado um arquivo csv de 10000 com os embeddings para utilizarmos a API no Python e fazer as requisições de fraude.

In [ ]:
# Título do Notebook
print("# Classificação de Análise de Fraude - Dataset Reduzido")

# Importação das Bibliotecas
print("## Importação e Instalação das Bibliotecas")
!pip install transformers
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModel

# Leitura e Redução do Dataset
print("## Leitura e Pré-processamento dos Dados")
# Lendo o arquivo original
dataframe_original = pd.read_csv('dados_dentista_large_dataset_150000_detailed.csv')
# Reduzindo para 10.000 linhas aleatórias
dataframe = dataframe_original.sample(n=10000, random_state=42)  # random_state para reprodutibilidade
print(f"Dataset reduzido para {len(dataframe)} linhas")
dataframe.head()

# Verificando as informações do dataset reduzido
dataframe.info()

# Configuração do Dispositivo e Modelo BERT
print("## Tokenizer e Modelo BERT")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

# Carregar o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased").to(device)

# Função para gerar embeddings em lotes
def gerar_embeddings_lote(textos):
    inputs = tokenizer(textos, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embeddings

# Transformando as Colunas em Embeddings
print("## Transformando as Colunas em Dados Vetoriais")
# Colunas de texto a serem transformadas
colunas_texto = ['sintomas', 'diagnostico', 'procedimento', 'lista_de_insumos']

# Processando em lotes para eficiência (tamanho do lote ajustado para dataset menor)
tamanho_lote = 32
for coluna in colunas_texto:
    print(f"Gerando embeddings para a coluna: {coluna}")
    embeddings = []
    for i in range(0, len(dataframe), tamanho_lote):
        lote = dataframe[coluna].iloc[i:i+tamanho_lote].tolist()
        embeddings_lote = gerar_embeddings_lote(lote)
        embeddings.extend(embeddings_lote)
    dataframe[f"{coluna}_embedding"] = embeddings

# Visualizando o resultado
print("Amostra do dataset com embeddings:")
dataframe.sample(5)

# Salvando o Dataset Reduzido (opcional)
dataframe.to_csv('dados_dentista_reduzido_10000.csv', index=False)
print("Dataset reduzido salvo como 'dados_dentista_reduzido_10000.csv'")

# (Opcional) Próximos passos: Treinamento ou Teste com API
print("## Próximos Passos")
print("Aqui você pode adicionar o código para treinar o modelo ou testar com a API.")
# Exemplo de placeholder para integração com API
# import requests
# url_api = "sua_url_api_aqui"
# response = requests.post(url_api, json=dataframe.to_dict())
# print(response.json())

# Classificação de Análise de Fraude - Dataset Reduzido
## Importação e Instalação das Bibliotecas
## Leitura e Pré-processamento dos Dados
Dataset reduzido para 10000 linhas
<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 59770 to 93473
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id_do_dentista             10000 non-null  float64
 1   id_do_segurado             10000 non-null  float64
 2   sintomas                   10000 non-null  object 
 3   diagnostico                10000 non-null  object 
 4   procedimento               10000 non-null  object 
 5   lista_de_insumos           10000 non-null  object 
 6   quantidade_de_cada_insumo  10000 non-null  object 
 7   chance_de_fraude           10000 non-null  float64
dtypes: float64(3), object(5)
memory usage: 703.1+ KB
## Tokenizer e Modelo BERT
Usando o dispositivo: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## Transformando as Colunas em Dados Vetoriais
Gerando embeddings para a coluna: sintomas
Gerando embeddings para a coluna: diagnostico
Gerando embeddings para a coluna: procedimento
Gerando embeddings para a coluna: lista_de_insumos
Amostra do dataset com embeddings:
Dataset reduzido salvo como 'dados_dentista_reduzido_10000.csv'
## Próximos Passos
Aqui você pode adicionar o código para treinar o modelo ou testar com a API.
